## 2. Die gesammelten Daten analysieren


Die Daten sind erstmals nur das: Daten. Wir müssen also noch herausfinden, was für Eigenschaften diese überhaupt haben, um sie später verwenden zu können.


Zuerst wieder die Bibliotheken und Konfigurationen...


In [ ]:
import json
from collections import defaultdict


mode = 1
# 1 = Nationalratswahlen
# 2 = Ständeratswahlen

filename = "nationalrat_raw.json" if mode == 1 else "ständerat_raw.json"

Hier ist noch eine kleine Hilfsfunktion zum schönen Darstellen einiger Daten


In [ ]:
def print_dict(d: dict):
    maxlen_keys = max(map(len, map(str, d.keys())))
    maxlen_values = max(map(len, map(str, d.values())))
    for key in d.keys():
        # abc:.........420
        print(f"{key:.<{maxlen_keys}}{d[key]:.>{maxlen_values+1}}")

Dann folgt das Laden der Daten.


In [ ]:
candidates = json.load(open(filename, encoding="utf-8"))

Wie viele Personen kandidieren für jede Partei?


In [ ]:
party_count = defaultdict(int)

for candidate in candidates:
    party = candidate.get("partyAbbreviation")
    if party:
        party_count[party] += 1

# Sortieren nach Anzahl Kandidierende pro Partei
party_count = dict(sorted(party_count.items(), key=lambda item: item[1], reverse=True))
print_dict(party_count)

Anscheinend gibt es einige Parteien, die nur sehr wenige Mitglieder haben. Beim späteren Bereinigen der Daten müssen wir Kandidierende dieser Parteien leider aussortieren, da eine gewisse Menge an Daten erforderlich ist, um ein neuronales Netzwerk zu trainieren.


Wie sieht's aus mit dem Frauenanteil der Kandidierenden?


In [ ]:
# Ein Dictionary für Frauen und Männer, eins nur für Frauen. Kandidierende ohne Eintrag beim Feld "gender" werden ignoriert
f_and_m = defaultdict(int)
f_only = defaultdict(int)

for candidate in candidates:
    party = candidate.get("partyAbbreviation")
    gender = candidate.get("gender")
    if party:
        if gender == "f":
            f_only[party] += 1
            f_and_m[party] += 1
        if gender == "m":
            f_and_m[party] += 1

# Berechnet den Frauenanteil
female_share = dict(
    [
        (party_name, f"{ratio:.0%}")
        for party_name, ratio in sorted(
            [
                (party_name, women / f_and_m[party_name])
                for party_name, women in f_only.items()
            ],
            key=lambda item: item[1],
            reverse=True,
        )
    ]
)

print_dict(female_share)